In [1]:
import json
import string

In [2]:
vocabs_pool = set()
with open(r"vocabs\en-len5-easy.json") as file:
    json_data = json.load(file)
    for d in json_data:
        vocabs_pool.add(d)

num_vocabs = len(vocabs_pool)
alphabets = string.ascii_lowercase

In [3]:
def change_character(vocab: str) -> set[str]:
    new_vocabs = set()
    for i, c in enumerate(vocab):
        for a in alphabets:
            if a != c:
                new_vocab = vocab[:i] + a + vocab[i + 1 :]
                new_vocabs.add(new_vocab)

    return new_vocabs


print(change_character("abc"))

{'avc', 'alc', 'abq', 'azc', 'adc', 'auc', 'abj', 'abt', 'awc', 'aec', 'abl', 'tbc', 'sbc', 'abi', 'aby', 'asc', 'agc', 'abn', 'kbc', 'axc', 'abo', 'ebc', 'abx', 'wbc', 'aqc', 'akc', 'arc', 'dbc', 'qbc', 'amc', 'anc', 'abf', 'pbc', 'abh', 'aba', 'abs', 'abz', 'gbc', 'abr', 'jbc', 'abu', 'obc', 'ajc', 'apc', 'abk', 'ybc', 'xbc', 'zbc', 'ayc', 'abb', 'abd', 'rbc', 'abp', 'aac', 'abv', 'hbc', 'ubc', 'abw', 'ahc', 'bbc', 'nbc', 'aic', 'acc', 'fbc', 'cbc', 'afc', 'aoc', 'abg', 'mbc', 'abe', 'vbc', 'atc', 'abm', 'lbc', 'ibc'}


In [4]:
def distance(from_: str, to: str, max_distance=None) -> tuple[int, list[str]]:
    vocab_path = {from_: ""}
    queue = [(from_, 0)]
    while queue:
        (cur_vocab, cur_distance) = queue.pop(0)
        if cur_vocab == to:
            path = [cur_vocab]
            while path[0] != from_:
                path.insert(0, vocab_path[path[0]])
            return (cur_distance, path)

        if cur_distance == max_distance:
            continue

        for new_vocab in change_character(cur_vocab):
            if (new_vocab not in vocab_path) and (new_vocab in vocabs_pool):
                vocab_path[new_vocab] = cur_vocab
                queue.append((new_vocab, cur_distance + 1))

    return (-1, [])

In [17]:
print(distance("admin", "admit"))

(1, ['admin', 'admit'])


In [9]:
max_distance = 6
count = 0
vocabs_involved = set()
for i, v1 in enumerate(vocabs_pool):
    for j, v2 in enumerate(vocabs_pool):
        dist = distance(v1, v2, max_distance=max_distance)
        if dist[0] > 0:
            count += 1
            vocabs_involved = vocabs_involved.union(set(dist[1]))

    if i % (num_vocabs // 10) == 0:
        print(f'checking "{v1}"| {i}/{num_vocabs}')

pair_rate = count / (num_vocabs * (num_vocabs - 1))
print(f"pair rate: {count}/{num_vocabs*(num_vocabs-1)} = {pair_rate:.2%}")

participation_rate = len(vocabs_involved) / num_vocabs
print(f"participation: {len(vocabs_involved)}/{num_vocabs} = {participation_rate:.2%}")

checking "autos"| 0/1000
checking "banks"| 100/1000
checking "charm"| 200/1000
checking "above"| 300/1000
checking "rally"| 400/1000
checking "blair"| 500/1000
checking "picks"| 600/1000
checking "order"| 700/1000
checking "pairs"| 800/1000
checking "songs"| 900/1000
pair rate: 2080/999000 = 207792.00%
participation: 542/1000 = 54.20%


In [7]:
def create_distance_graph():
    distance_graph: dict[tuple[str, str], tuple[int, list[str]]] = {}

    def calculate_distance(from_: str, to: str):
        if (from_, to) in distance_graph:
            return

        min_distance = -1
        min_path = []
        for new_vocab in change_character(from_):
            if new_vocab in vocabs_pool:
                calculate_distance(new_vocab, to)
                distance, path = distance_graph[(new_vocab, to)]
                if min_distance == -1 or 0 <= distance < min_distance:
                    min_distance = distance + 1
                    min_path = [from_] + path
        distance_graph[(from_, to)] = (min_distance, min_path)

    for v1 in vocabs_pool:
        for v2 in vocabs_pool:
            calculate_distance(v1, v2)
    return distance_graph


# distance_graph = create_distance_graph()
# print(distance_graph)